In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function, absolute_import, division

In [ ]:
import numpy as np
from wgomoku import GomokuBoard, GomokuTools as gt, NH9x9, Heuristics

In [ ]:
def num_offensive(o, d):
    s, l, offset = gt.mask2(o, d)
    m2o_bits = gt.as_bit_array(s)[:l]
    max_count = 0
    for w in [2,1,0]:
        i = 0
        while i <= len(m2o_bits) - 2 - w:
            count = sum(m2o_bits[i:i+w+2])
            count = 3*count - (w+2)
            if count > max_count:
                max_count = count
            i+=1
    if m2o_bits[0] == 0:
        max_count += 1.5
    if m2o_bits[-1] == 0:
        max_count += 1.5

    return max_count        


In [ ]:
def line_score(xo):
    o,d = gt.line_for_xo(xo)
    m = gt.mask(o,d)
    m2 = gt.mask2(o,d)
    if m2[1] >= 4 and sum(gt.as_bit_array(m2[0])) >= 1:
        return num_offensive(o,d) - 2
    else:
        return 0

In [ ]:
for i in [
    '...ox...',
    '...x..xo',
    '.....x..',
    '...x.xo.',
    '...x..x.',
    '...xxo..',
    'xx......',
    '...x.x..',
    '..oxx.x.',
    '...xx...',
    '.xx.....',
    '...xxxo.',
    '...xx.x.',
    '...xxx..',
    '.oxxxx..',
    '..xxxx..',
]: 
    print(i, line_score(i))

In [ ]:
kappa=3.0
def nhcombine(l):
    l_ = sorted(l)
    
    if l_[-1]>7:
        return 8 # Done
        
    if l_[-1]==7 or (l_[-1] in [4.5,5.5,6,6.5,7.0] and l_[-2] >= 4):
        return 7 # truly strong
    
    if l_[-1] in [4,5] and l_[-2] in [4,5]:
        return 6.9 # can only be countered by strong counter-attack
    
    return (l_[-1]**kappa + l_[-2]**kappa)**(1/kappa)

In [ ]:
values = np.arange(20)/2
values

In [ ]:
precomputed = np.zeros([160000])
for e in values:
    for ne in values:
        for n in values:
            for nw in values:
                v = nhcombine([e, ne, n, nw])
                precomputed[int(2*(8000*e+400*ne+20*n+nw))]=v

In [ ]:
line_counts=9-(np.sqrt(np.random.randint(0,18**2, [5, 5, 4]))).astype(int)/2

In [ ]:
e,ne,n,nw = np.rollaxis(line_counts, 2, 0)
indices = (2*(8000*e+400*ne+20*n+nw)).astype(int)
scores = precomputed[indices]

In [ ]:
for i in range(5):
    for j in range(5):
        print(line_counts[i][j], scores[i][j])

In [ ]:
nhcombine([6.5, 0.,  2.5, 3.5 ])

Now, a classical strategy could be: (A:attack, D:defend)
- Win with level A7+ position
- if not possible, ALWAYS defend single level D7+ position
- Give up if more than 1 D7+ position exist. 

- ALWAYS attack at level A7 position
- if not possible, ALWAYS defend level D7 position

- ALWAYS tree search level A6.9 for a strong defense
    - if not a strong defense, attack at that position
    - if strong defense possible, consider deeper tree search or alternatives
- ALWAYS tree search level D6.9 for a strong counter-attack
    - if no strong counter-attack possible, defend that position
    - if strong counter-attack possible, consider deeper tree search or alternatives
- Choose greedy (when fighting) or e-greedy when learning
    - allow policies with more agressive or more defensive style
